In [ ]:
import sys
sys.path.append('c:\\Users\\Admin\\few_shot_learning\\')

from Utils.Class import Predictor, FSL, Random, Nearest_Neighbour
from Utils.saveLoad import save_data, load_data
from Utils import testModels, buildModel, utils

import preprocess

import numpy as np
import os
import random

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split


Get all the file names into a dict, where the key is the class name and value is a list of filenames for that class

In [ ]:
missile_dir = r'C:\Users\Admin\few_shot_learning\missile\data\missile'
missiles = os.listdir(missile_dir)
all_imgs_dir = {}

for missile in missiles:
    imgs_dir = r'{}\{}'.format(missile_dir, missile)
    imgs = os.listdir(imgs_dir)
    imgs = [r'{}\{}'.format(imgs_dir, fn.strip()) for fn in imgs]
    all_imgs_dir.update({missile : imgs})

In [ ]:
uav_dir = r'C:\Users\Admin\few_shot_learning\missile\data\uav'
uavs = os.listdir(uav_dir)

for uav in uavs:
    imgs_dir = r'{}\{}'.format(uav_dir, uav)
    imgs = os.listdir(imgs_dir)
    imgs = [r'{}\{}'.format(imgs_dir, fn.strip()) for fn in imgs]
    all_imgs_dir.update({uav : imgs})

In [ ]:
print(list(all_imgs_dir.keys()))
print(all_imgs_dir['aster'][0])

ensure our filenames have been compiled correctly, and test our pad_and_resize function

In [ ]:
img = plt.imread(all_imgs_dir['aster'][5])
plt.figure()
plt.imshow(img)
print(img.shape)

plt.figure()
new_img = preprocess.pad_and_resize(img, desired_ratio=1.4, width=280, height=200)
plt.imshow(new_img)
print(new_img.shape)
print(type(new_img))

load the preprocessed data that we require (if any)

In [ ]:
kwargs = {'X' : True,
          'Y' : True,
          # 'all_pairs' : True,
          # 'all_labels' : True,
          # 'all_raw_labels' : True,
          'train_pairs' : True,
          'train_labels' : True,
          'val_pairs' : True,
          'val_labels' : True,
        #   'test_pairs': True,
        #   'test_labels' : True
        
        }
        
data = load_data(path='preprocess\\', **kwargs)


preprocessing stuff

In [ ]:
X, Y = preprocess.get_all_X_Y(all_imgs_dir, desired_ratio=1.4, width=280, height=200)

In [ ]:
pos_pairs, pos_labels, pos_raw_labels = preprocess.make_positive_pairs(data['X'], data['Y'], list(all_imgs_dir.keys()), N=10000)

In [ ]:
neg_pairs, neg_labels, neg_raw_labels = preprocess.make_negative_pairs(data['X'], data['Y'], N=10000)

In [ ]:
utils.show_pairs(5, data['all_pairs'], data['all_labels'], data['all_raw_labels'], random=True)

split our dataset twice to get training, testing and validation sets

In [ ]:
all_train_pairs, test_pairs, all_train_labels, test_labels = train_test_split(data['all_pairs'], data['all_labels'], test_size=0.2, random_state=42)

train_pairs, val_pairs, train_labels, val_labels = train_test_split(all_train_pairs, all_train_labels, test_size=0.2, random_state=50)

print(train_pairs.shape, val_pairs.shape, test_pairs.shape)

We can create a new prototype model to train

In [ ]:
import keras
import tensorflow as tf

from keras.models import load_model

from keras.callbacks import History 

from keras.optimizers import Adam
from keras import backend as K

In [ ]:
new_model = buildModel.get_pretrained_model((200, 280, 3))
new_model.summary()

In [ ]:
val_inputs = [data['val_pairs'][:,0], data['val_pairs'][:,1]]

In [ ]:
train_inputs = [data['train_pairs'][:,0], data['train_pairs'][:,1]]

In [ ]:
optimizer = Adam(lr = 0.0001)
new_model.compile(loss = "binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
hist = History()

new_model.fit(train_inputs, data['train_labels'][:], batch_size=64, epochs=5, callbacks=[buildModel.EarlyStoppingAtMinLoss(patience=1, val_inputs=val_inputs, val_labels=data['val_labels'][:]), hist])

Visualise its learning curve

In [ ]:
with open('plots/LC_vgg16_dict.txt', 'r') as f:
    contents = f.read()
    hist_dict = eval(contents)
# for k, v in hist.history.items():
#     hist_dict[k] = (hist_dict[k] + hist.history[k])[:]

In [ ]:
X = range(1, len(hist_dict['loss']) + 1)
for k, v in hist_dict.items():
    if 'loss' in k:
        plt.plot(X, v, label=k)

plt.xlabel('Epoch')
plt.xticks(range(0, len(hist_dict['loss']) + 1, 5))
plt.xlim(X[0], X[-1]+1)

plt.ylabel('Mean Loss')
plt.legend()
# plt.savefig('plots/LC_vgg16_missile.pdf')
plt.show()
    

Or we can load a saved model for evaluation or to continue training

In [ ]:
model = load_model('models\\vgg16_missile_early_stop')

In [ ]:
N = range(1,5)
NUM_TESTS = 10
accuracies = testModels.get_accuracy([FSL(model, name='FSL (Pretrained) (Early Stop)')], data['X'], data['Y'], N, NUM_TESTS)

In [ ]:
testModels.plot_accuracy(accuracies)

In [ ]:
test_inputs = [data['test_pairs'][:,0], data['test_pairs'][:,1]]

In [ ]:
model.evaluate(x=test_inputs, y=data['test_labels'])